In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [106]:
import numpy as np
import pandas as pd
# update 순서
# df = pd.read_csv('./kaggle/input/spaceship-titanic/train_Destination_update.csv')

# Age 61세보다 낮은 값의 중앙값으로 처리
# df = pd.read_csv('./kaggle/input/spaceship-titanic/train_Age_update.csv')

# score 0.6649279264211672
# df = pd.read_csv('./kaggle/input/spaceship-titanic/train_VRDeck_update.csv')

# score 0.6853975923245519 FoodCourt
# df = pd.read_csv('./kaggle/input/spaceship-titanic/train_FoodCourt_update.csv')


# ShoppingMall 0.5775638542322463
# df = pd.read_csv('./kaggle/input/spaceship-titanic/train_ShoppingMall_update.csv')


# train_RoomService_update 0.5751302859250393
# df = pd.read_csv('./kaggle/input/spaceship-titanic/train_RoomService_update.csv')

# Spa 0.6491947772592053
# df = pd.read_csv('./kaggle/input/spaceship-titanic/train_Spa_update.csv')

df = pd.read_csv('./kaggle/input/spaceship-titanic/train_Spa_update.csv')
df_train = df


In [107]:
df_train.isnull().sum()

PassengerId       0
HomePlanet        0
CryoSleep         0
Cabin_A           0
Cabin_B           0
Cabin_C           0
Cabin_D           0
Cabin_E           0
Cabin_F           0
Cabin_G           0
Cabin_T           0
Cabin_num         0
Cabin_port        0
Destination       0
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Name            198
Transported       0
dtype: int64

In [102]:
df_train.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin_A', 'Cabin_B',
       'Cabin_C', 'Cabin_D', 'Cabin_E', 'Cabin_F', 'Cabin_G', 'Cabin_T',
       'Cabin_num', 'Cabin_port', 'Destination', 'Age', 'VIP', 'RoomService',
       'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name', 'Transported'],
      dtype='object')

In [103]:
# train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBRegressor


def fillnull(column_name):

    global X_train, X_test, y_train, y_test

    # df_FoodCourt =  df_train중에서 FoodCourt null값이 존재하는 열만 추출 
    # -> 훈련된 모델의 값으로 다시 채울 예정
    df_column_name = df_train[df_train[column_name].isnull()]
    #df_train 중에서 FoodCourt 의 null값이 존재하는 열만 추출 -> 추후 훈련할 데이터프레임
    df_train01 = df_train.dropna(subset=[column_name])
    # df_train
    df_train[df_train[column_name].isnull()]

    X_train, X_test, y_train, y_test = train_test_split(df_train01.drop(['PassengerId',column_name,'Name'],axis=1),df_train01[column_name], test_size=0.2, random_state=42)

    # scaler
    scaler = StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    # 모델
    model = xgb.XGBRegressor(early_stopping_rounds=10)
    eval_set = [(X_test, y_test)]
    model.fit(X_train, y_train, eval_set=eval_set, verbose=False)#, verbose=True

    pred = model.predict(X_test)
    score = model.score(X_train,y_train)
    print(score)

    global df_update
    # df_01 은 df_train 중에서 FoodCourt 의 값이 null인 경우의 데이터프레임
    df_column_name_null = df_column_name.drop(column_name,axis=1)
    # 훈련된 xgboost모델을 사용해서 df_homeplanet_null의 HomePlanet값을 예측
    pred = model.predict(df_column_name_null.drop(['PassengerId','Name'],axis=1))
    pred.max()
    pred = np.round(pred)

    # 예측된 값을 다시 df_homeplanet으로 삽입(null값 채우기)
    df_column_name[column_name] = pred
    df_column_name
    
    # null값을 채운 dataframe과 df_train01 합치기 
    df_update = pd.concat([df_train01,df_column_name],axis=0)

fillnull('Spa')


0.6491947772592053


C:\Users\deepBlue\AppData\Local\Temp\ipykernel_11808\694111669.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_column_name[column_name] = pred


In [104]:
df_update[df_update['RoomService'].isnull()]

,PassengerId,HomePlanet,CryoSleep,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,...,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported


In [89]:
df_update.isnull().sum()

PassengerId       0
HomePlanet        0
CryoSleep         0
Cabin_A           0
Cabin_B           0
Cabin_C           0
Cabin_D           0
Cabin_E           0
Cabin_F           0
Cabin_G           0
Cabin_T           0
Cabin_num         0
Cabin_port        0
Destination       0
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Name            198
Transported       0
dtype: int64

In [105]:
# df_update.to_csv('./kaggle/input/spaceship-titanic/train_Spa_update.csv', index=False)